# Text analytics

## Datensammlung

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.common.action_chains import ActionChains  

import pyautogui  
import pandas as pd
import time
import datetime
import random
import re

#  In Terminal oder Anaconda Prompt:
#  pip install selenium, pyautogui

### Datensammlung: Artikel

In [70]:
dax_companies = {
    36714349 : "Adidas",        # N
    98641 : "Airbus Group (EADS)",        # N
    83219 : "Allianz",        # N
    34694526 : "BASF",        # N
    25272187 : "Bayer",        # N
    81500 : "Beiersdorf",        # N
    81490 : "BMW",        # N
    28975512 : "Brenntag",        # N
    83139 : "Commerzbank",        # N
    81967 : "Continental",        # N
    12587335 : "Covestro",        # L
    208894340 : "Daimler Truck",        # L
    81348 : "Deutsche Bank",        # L
    3459922 : "Deutsche Börse",        # L
    181029 : "Deutsche Telekom",        # L
    82088 : "DHL Group (ex Deutsche Post)",        # L
    21074892 : "E.ON",        # L
    82235 : "Fresenius",        # L
    182351 : "Hannover Rück",        # L
    82340 : "Heidelberg Materials",        # L
    82344 : "Henkel",        # J
    82561 : "Infineon",        # J
    82840 : "Mercedes-Benz Group (Daimler)",        # J
    82676 : "Merck",        # J
    12254991 : "MTU Aero Engines",        # J
    83258 : "Münchener Rück",        # J
    231291652 : "Porsche AG (Vz.)",        # J
    21178031 : "Porsche Automobil Holding SE",        # J
    90268 : "Qiagen NV",        # J
    82811 : "Rheinmetall",        # J
    82818 : "RWE",        # M
    82849 : "SAP",        # M
    82852 : "Sartorius (Vz.)",        # M
    82902 : "Siemens",        # M
    180455076 : "Siemens Energy",        # M
    134397957 : "Siemens Healthineers",        # M
    15630917 : "Symrise",        # M
    83057 : "Volkswagen (Vz.)",        # M
    62903083 : "Vonovia",        # M
    81388537 : "Zalando"        # M
          }

news_subtypes = {
    1 : "Aktieneinstufung",
    2 : "Analyse",
    3 : "Directors Dealings",
    4 : "Empfehlung",
    5 : "Erklärstück",
    6 : "Fundamentalanalyse",
    7 : "Interview",
    8 : "Kommentar",
    9 : "Marktbericht",
    10 : "News", 
    11 : "Pflichtmitteilung",
    12 : "Pressemitteilung",
    13 : "Ratgeber",
    14 : "Technische Analyse",
    15 : "Themen-Spezial"
}

article_data = []

filtered_words = ["Top Aktien", "Top Märkte", "Ratgeber", "onvista media GmbH", "Rechtliche Hinweise", "App", "Social Media", "Veröffentlichung der Original-Studie", "Erstmalige Weitergabe der Original-Studie"]  # Wörter die in der HTML Struktur als Paragraphen gekennzeichnet werden, aber nicht interessant für die Analyse sind

detailedFeedback = True  # True wenn Titel, Datum, Quelle, URL und Inhalt jedes Artikel als feedback im Terminal angezeigt werden soll. Bei False wird nur genereller Fortschritt ausgegeben.

driver = webdriver.Firefox()
driver.maximize_window()

# Iteration über alle keys der Unternehmen aus dem dictionary 'dax_companies'
for key in dax_companies.keys():  
    
    # Iteration je Unternehmen über alle 15 Newstypen
    for subtype in news_subtypes.keys():  
        
        # Iteration je Unternehmen und Subtyp über max. 10 Seitenzahlen
        for page in range(0,9):  

            driver.get(f'https://www.onvista.de/news/finder?page={page}&entityType=STOCK&entityValue={key}&idSubTypeGroups={subtype}')  # gefilterte Seiten lassen sich über den unternehmensspezifischen key aufrufen
            
            time.sleep(random.uniform(4,5))

            # Cookie Banner Handling
            try:
                iframe = driver.find_element(By.CSS_SELECTOR, "#sp_message_container_800509 > iframe")  # id des Divs über dem Iframe, das den Button auf des Cookie-Pop-up enthält
                driver.switch_to.frame(iframe)  # Wechselt den Fokus des Webdrivers auf den Inhalt des iframes
                
                button = driver.find_element(By.XPATH, "//button[@class='message-component message-button no-children focusable sp_choice_type_13']").click()
                driver.switch_to.default_content()    # Wechselt den Fokus des Webdrivers auf den Inhalt der Webseite

                #pyautogui.moveTo(823,359)  # Bewegt Maus zur angegebenen Koordiate. Workaround für Cookie-Banner (pyautogui.position() gibt position der Maus aus)
                #pyautogui.click()

            except:
                pass
            
            
            # Artikelübersicht
            try:
                articles = driver.find_elements(By.XPATH,'//strong[@class="ov-display--block ov-word-wrap"]')  # Beinhaltet alle Artikel der aktuellen Seite
                counter = 0  # Counter um feedback über Fortschritt des Scraping zu geben.

                if len(articles) == 0:  #Überprüft ob aktuelle Seite der Artikelübersicht leer ist.
                    print("#"*100)
                    print(f"[o] Auf Seite {page+1} wurden keine Artikel gefunden.")
                    print("#"*100)

                    break  # Unterbricht die Iteration über die Seitenzahl

                else:
                    print("#"*100)
                    print(f"[+] Es wurden {len(articles)} Artikel für {dax_companies[key]} zum Newstyp '{news_subtypes[subtype]}' auf Seite {page+1} gefunden.")
                    print("#"*100)

            except:
                
                pass

            # Iteration über alle gefundenen Artikel
            for article in articles:
            
                article.location_once_scrolled_into_view  # Automatisches Scrollen zu jedem Artikel
                time.sleep(random.uniform(1, 2.5))

                actions = ActionChains(driver)
                actions.key_down(Keys.CONTROL).click(article).key_up(Keys.CONTROL).perform()  # Öffnet Artikel in einem neuen Tab

                driver.switch_to.window(driver.window_handles[1])  # Fokus des Webdrivers wird auf neuen Tab gerichtet

                time.sleep(random.uniform(5.1, 7.4))  # Wartezeit um sicher zu gehen, dass Seite vollständig geladen wurde sowie um natürliches Nutzerverhalten zu simulieren


                # Umgang mit Werbung
                try:
                    ad = driver.find_element(By.XPATH, '//div[@class="flex-layout flex-layout__align-items--center flex-layout__justify-content--end text-size--xsmall ov-subline"]').text.strip()  # Extraktion dem Titel aus der HTML-Struktur, teweilweise problematisch zu erkennen
                    ad_text = ad.split('\n')

                    print(ad_text[0])

                    if "Werbung" in ad_text[0]:  # Umgang mit Werbung die direkt auf onvista.de dargestellt wird
                        driver.close()
                        driver.switch_to.window(driver.window_handles[0])  # Fokus zurück zum ersten Tab
                        continue
                    
                    #ad_yt = driver.find_element(By.XPATH, '//h1[@class="SGW9xe"]').text.strip()  # XPATH zu Cookie-Banner von Youtube
                    #ad_yt_text = ad.split('\n')

                    #print(ad_yt_text)

                    #if "YouTube" in ad_yt_text[0]:
                    #    driver.close()
                    #    driver.switch_to.window(driver.window_handles[0])
                    #    continue

                    current_url = driver.current_url
                    if "onvista.de" not in current_url:  # Umgang mit Werbung die über ein Element aus der Artikelübersicht zu einer Webseite außerhalb von onvista führt
                        driver.close()
                        driver.switch_to.window(driver.window_handles[0])
                        continue

                except:
                    pass
                
                
                # Extraktion des Titels
                try:   
                    article_title = driver.find_element(By.XPATH, '//h1[@class="headline headline--h1 headline--full-width outer-spacing--xxsmall-bottom"]').text.strip() 
                    title = article_title.split('\n')[0]
                except:
                    title = "NO_TITLE"


                # Extraktion des Datums und Uhrzeit der Veröffentlichung
                try:
                    article_date = driver.find_element(By.XPATH, '//time[@class="color--cd-anthracite"]').text.strip()
                    date = article_date.split('\n')[0]

                except:
                    date = "NO_DATE"


                # Extraktion der Quelle
                try:
                    article_source = driver.find_element(By.XPATH, '//a[@class="link link--unstyled link--underline"]').text.strip()
                    source = article_source.split('\n')[0]
                
                except:
                    source = "NO_SOURCE"


                # Extrakktion der Text-Inhalte. Je nach Artikelstruktur werden verschiedene Ansätze zur Sammlung der Inhalte der Paragraphen benötigt:
                try:  
                    

                    if "EQS" in source:  # primäre Quelle für öffentliche Bekanntgaben, veröffentlicht in Form von Tabellen

                        tables = driver.find_elements(By.TAG_NAME, "table")

                        text = tables[0].text

                    else :  # Restliche Artikel in Paragraphen

                        article_text = driver.find_elements(By.TAG_NAME,'p') # Enthält alle Paragraphen des aktuellen Artikels. Inhalt des Artikel ist am besten über die Paragraphen ansteuerbar.
                        time.sleep(random.uniform(1.2, 2))

                        if article_text is None:  # Da teilweise nicht erkennbar, wird überprüft ob article_text leer ist, um die folgenden Ansätze zu versuchen

                            article_text = driver.find_elements(By.CSS_SELECTOR, 'p.paragraph')  

                        elif article_text is None:

                            article_text = driver.find_elements(By.XPATH, '//p[@class="paragraph Styles_ov-content-item-list__item__wK2EM"]') 

                        text_list = []

                        for paragraph in article_text:  # Iteration über alle enthaltenen Paragraphen
                            
                            paragraph.location_once_scrolled_into_view
                            time.sleep(0.5)
                            text_list.append(paragraph.text)  
                            
                            for element in text_list[-9:]:  # Die letzten 9 Elemente sind meist/immer die in 'filtered_words' vorkommende Wörter, welche laut HTML-Struktur jedoch als paragraph deklariert wurden.

                                if element in filtered_words:
                                    text_list.remove(element) 

                            text = '\n'.join(text_list)  # Aneinanderfügung der Paragraphen zu einem String, welcher durch Zeilenumbrüche (\n) getrennet wird.

                except:
                    text = "NO_TEXT"

                url = driver.current_url

                data = {"Unternehmen" : dax_companies[key], "Newstyp" : news_subtypes[subtype],  "Titel" : title, "Datum" : date, "Quelle" : source, "URL" : url, "Text" : text}  # Zusammenführung der gesammelten Daten des Artikels

                article_data.append(data)

                counter += 1

                print("-"*100)
                print(f"[*] Artikel {counter} / {len(articles)} auf Seite {page+1} überprüft.")
                print("-"*100)

                if detailedFeedback:
                    print(f"Unternehmen: {dax_companies[key]}")
                    print(f"Newstyp: {news_subtypes[subtype]}")
                    print(f"Titel: {title}")
                    print(f"Datum: {date}")
                    print(f"Quelle: {source}")
                    print(f"URL: {url}")
                    print(f"Inhalt:\n{text}")
                    print("-"*100)

                driver.close()  # Tab schließen
                driver.switch_to.window(driver.window_handles[0])  # Fokus zurück zum ersten Tab mit der Übersicht der Artikel

driver.close()

now = datetime.datetime.now()  # Zum Anfügen des aktuellen Datum/Uhrzeit an Dateinamen
date_time_str = now.strftime('%Y%m%d_%H%M%S')
date_time_str

df = pd.DataFrame(article_data)

df.to_excel(f'data/Onvista_Articles-{date_time_str}.xlsx', index=False)
df.to_csv(f'data/Onvista_Articles-{date_time_str}.csv', index=False)

####################################################################################################
[+] Es wurden 52 Artikel für Zalando zum Newstyp 'Aktieneinstufung' auf Seite 1 gefunden.
####################################################################################################
----------------------------------------------------------------------------------------------------
[*] Artikel 1 / 52 auf Seite 1 überprüft.
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
[*] Artikel 2 / 52 auf Seite 1 überprüft.
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
[*] Artikel 3 / 52 auf Seite 1 überprüft.
-----------------------------------------------------------------------------

In [69]:
df = pd.read_csv('Onvista_Articles.csv')
len(df)

175

### Datensammlung: Historische Börsenkurse